<a href="https://colab.research.google.com/github/lee-euijin/AppleFarm/blob/%EC%83%81%EC%A4%80/Model/skeleton_preprocessing_and_Model_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data upload

In [2]:
# mount Google Drive
from google.colab import drive
import os
drive.mount('/content/gdrive')

# unzip
import zipfile, shutil

dataset = '/content/gdrive/My Drive/학교 수업 자료/딥러닝/Project/static_data.zip'
dst_path = '/content/static_data'
dst_file = os.path.join(dst_path, 'static_data.zip')

if not os.path.exists(dst_path):
  os.makedirs(dst_path)

shutil.copy(dataset, dst_file)

with zipfile.ZipFile(dst_file, 'r') as file:
  file.extractall(dst_path)

Mounted at /content/gdrive


In [3]:
import os
imagepaths = []

for dirname, _, filenames in os.walk('/content/static_data'):
  for filename in filenames:
    path = os.path.join(dirname, filename)
    if path.endswith("png"):
      imagepaths.append(path)

print(len(imagepaths))
print(imagepaths[0])

40000
/content/static_data/leapgestrecog/leapGestRecog/02/07_ok/frame_02_07_0145.png


In [5]:
# google drive에 directory 추가
dir_path = '/content/gdrive/My Drive/학교 수업 자료/딥러닝/Project/static_skeleton_data'

for path in imagepaths[:19999]:
  splitted = path.split("/")

  # 없으면 directory 추가
  dir = os.path.join(dir_path, splitted[5])

  if not os.path.exists(dir):
    os.makedirs(dir)

  dir = os.path.join(dir_path, splitted[5], splitted[6])

  if not os.path.exists(dir):
    os.makedirs(dir)

## Data Preprocessing

In [6]:
!pip install mediapipe opencv-python

     |████████████████████████████████| 36.1MB 83kB/s 


### Data Preprocessing and Saving

In [ ]:
# 최초 save 이후에는 이 셀 실행하지 않음!
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import pandas as pd
import mediapipe as mp
from google.colab.patches import cv2_imshow

new_path = []

# 저장을 위한 path 생성
for path in imagepaths[:19999]:
  splitted = path.split("/")
  temp = '/' + splitted[5] + '/' + splitted[6] + '/' + splitted[7]
  new_path.append(temp)

# train data preprocessing and save
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx in range(19999):
    # read an image, flip it around y-axis for correct handedness output
    image = cv.imread(imagepaths[idx])
    # convert the bgr image to rgb before processing
    results = hands.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))

    # print handedness and draw hand landmarks on the image
    # print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()

    # landmark만 그릴 image
    landmark_image = np.zeros(image.shape)
    for hand_landmarks in results.multi_hand_landmarks:
      '''print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height}'
      )'''
      mp_drawing.draw_landmarks(
          landmark_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    '''# imshow
    cv2_imshow(annotated_image)
    cv2_imshow(landmark_image)'''
    
    # save
    cv.imwrite(dir_path + new_path[idx], landmark_image)

IndexError: ignored

* index를 잘못 설정하여 에러가 발생했지만, 정상적으로 모두 저장되었다.

### path 확인

In [7]:
skeletonpaths = []

for dirname, _, filenames in os.walk(dir_path):
  for filename in filenames:
    path = os.path.join(dirname, filename)
    if path.endswith("png"):
      skeletonpaths.append(path)

print(len(skeletonpaths)) # 6766개 데이터 유실
print(skeletonpaths[0])

13234
/content/gdrive/My Drive/학교 수업 자료/딥러닝/Project/static_skeleton_data/00/05_thumb/frame_00_05_0113.png


## Training

In [10]:
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout

In [ ]:
# Creating trainig set and labels
X = []
y = []

for path in skeletonpaths[:13234]:
  # image data
  img = cv.imread(path)
  img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  img = cv.resize(img, (128, 128))

  X.append(img)

  # labels
  category = path.split("/")[9] # path에서 9번째가 label
  label = int(category.split("_")[0][1])
  y.append(label)

X = np.array(X)
X = X.reshape(len(skeletonpaths), 128, 128, 1)
y = np.array(y)

print("이미지 개수 : ", len(X))
print("라벨 개수 : ", len(y))
print(y[0], skeletonpaths[0])

In [ ]:
# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=10)

# model
model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(128,128,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=2, 
          validation_data=(X_test, y_test))

Epoch 1/5
166/166 - 41s - loss: 0.7203 - accuracy: 0.7850 - val_loss: 0.1907 - val_accuracy: 0.9411
Epoch 2/5
166/166 - 9s - loss: 0.1490 - accuracy: 0.9538 - val_loss: 0.1177 - val_accuracy: 0.9683
Epoch 3/5
166/166 - 9s - loss: 0.0971 - accuracy: 0.9708 - val_loss: 0.1074 - val_accuracy: 0.9679
Epoch 4/5
166/166 - 9s - loss: 0.0614 - accuracy: 0.9796 - val_loss: 0.1500 - val_accuracy: 0.9622
Epoch 5/5
166/166 - 9s - loss: 0.0498 - accuracy: 0.9830 - val_loss: 0.0906 - val_accuracy: 0.9781


In [ ]:
# plotting
loss = history.history['loss']
val_loss = history.history['val_loss']

eochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# model save
models.save('model.h5')